In [1]:
import  tensorflow as tf
import tensorflow.keras as k
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt 
import pickle
import os 

In [11]:
from read_images import read_prof_images

X, Y, class_labels = read_prof_images()
print(X.shape, Y.shape, class_labels)

(300, 256, 256, 3) (300,) ['gc', 'go', 'gt']


In [12]:
from read_images import read_our_radar_data

X, y, class_labels = read_our_radar_data()
print(X.shape, y.shape, class_labels)

(80, 64, 621) (80,) ['pinching', 'push', 'rolling_forward', 'swing']


In [27]:
X = np.abs(X)
X = np.expand_dims(X, axis=-1)
X.shape

(80, 64, 621, 1)

In [28]:
 from sklearn.model_selection import train_test_split

In [29]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.33, random_state=42)
y_train = tf.one_hot(y_train, y.shape[0])
y_test = tf.one_hot(y_test, y.shape[0])

In [30]:
model=k.Sequential()
model.add(tf.keras.Input(shape=X.shape[1:]))
model.add(k.layers.Conv2D(32,3,3,padding='valid',
    dilation_rate=(1, 1),
    activation="relu"))
model.add(k.layers.Flatten())
model.add(k.layers.Dense(64,activation="relu"))
model.add(k.layers.Dense(64,activation="relu"))
model.add(k.layers.Dense(64,activation="relu"))
model.add(k.layers.Dense(y.shape[0],activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 21, 207, 32)       320       
_________________________________________________________________
flatten_3 (Flatten)          (None, 139104)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8902720   
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 80)                5200      
Total params: 8,916,560
Trainable params: 8,916,560
Non-trainable params: 0
____________________________________________

In [32]:
history=model.fit(X_train,y_train,epochs=50,batch_size=8)

Epoch 1/50
7/7 [==============================] - 1s 55ms/step - loss: 102.4129 - accuracy: 0.1132
Epoch 2/50
7/7 [==============================] - 0s 53ms/step - loss: 24.4764 - accuracy: 0.4906
Epoch 3/50
7/7 [==============================] - 0s 55ms/step - loss: 9.3824 - accuracy: 0.6604
Epoch 4/50
7/7 [==============================] - 0s 48ms/step - loss: 8.1488 - accuracy: 0.5849
Epoch 5/50
7/7 [==============================] - 0s 51ms/step - loss: 1.3229 - accuracy: 0.9245
Epoch 6/50
7/7 [==============================] - 0s 51ms/step - loss: 0.9115 - accuracy: 0.9245
Epoch 7/50
7/7 [==============================] - 0s 57ms/step - loss: 0.0304 - accuracy: 0.9811
Epoch 8/50
7/7 [==============================] - 0s 52ms/step - loss: 7.7512e-04 - accuracy: 1.0000
Epoch 9/50
7/7 [==============================] - 0s 53ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 10/50
7/7 [==============================] - 0s 52ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 11/50
7/7 [=====

In [33]:
y_preds = model.predict(X_test)
y_preds

array([[7.93966574e-33, 1.00000000e+00, 4.31360660e-27, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 3.42858507e-14, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.83866051e-37, 1.00000000e+00, 1.33657514e-17, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.48860251e-05, 2.06270779e-04, 2.58302808e-01, ...,
        2.79643420e-33, 0.00000000e+00, 0.00000000e+00],
       [3.47380364e-07, 4.04548955e-05, 9.77949023e-01, ...,
        3.02990548e-37, 0.00000000e+00, 0.00000000e+00]], dtype=float32)

In [37]:
np.argmax(y_preds, axis=1)

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 3, 1, 3, 2, 1, 3, 2, 3, 0, 2,
       0, 2, 1, 3, 2], dtype=int64)

In [35]:
np.sum(y_preds - y_test)

4.3581757e-08

In [39]:
np.sum(np.where(np.argmax(y_preds, axis=1)==np.argmax(y_test, axis=1), 0, 1))

5

In [40]:
np.argmax(y_preds, axis=1)

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 3, 1, 3, 2, 1, 3, 2, 3, 0, 2,
       0, 2, 1, 3, 2], dtype=int64)

In [41]:
np.argmax(y_test, axis=1)

array([1, 0, 1, 1, 0, 1, 0, 3, 0, 0, 2, 1, 3, 1, 3, 2, 3, 3, 2, 2, 0, 3,
       0, 2, 1, 3, 2], dtype=int64)

In [11]:
X_test.shape

(99, 256, 256, 3)

In [12]:
X_test.shape

(99, 256, 256, 3)